# Greedy Algorithm

## Setup

In [ ]:
from graphs_2d import *

## Graph Definition

In [ ]:
# Generate a single graph
rows, cols = 5, 5
grid_graph = generate_2D_grid_graph(rows, cols)
adjacency_matrix = nx.adjacency_matrix(grid_graph).todense()

plot_graph(grid_graph)

type(grid_graph)

## Starting Node Selection

### Weighted Select

In [ ]:
# Run the selection 10000 times
counts = np.zeros(len(grid_graph.nodes()))
for _ in range(100000):
    node = select_starting_node_8way_probablistic(grid_graph)
    node_index = list(grid_graph.nodes()).index(node)
    counts[node_index] += 1

# Reshape counts to match the grid for visualization
counts_matrix = counts.reshape((rows, cols))

plt.figure(figsize=(6, 5))
plt.imshow(counts_matrix, cmap='viridis', interpolation='nearest')
plt.colorbar(label='Number of times selected')
plt.title('Node Selection Frequency (100000 trials)')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

### Equal Select From Highest Neighbors

In [ ]:
# Run the selection 10000 times
counts = np.zeros(len(grid_graph.nodes()))
for _ in range(100000):
    node = select_starting_node_max_neighbors(grid_graph)
    node_index = list(grid_graph.nodes()).index(node)
    counts[node_index] += 1

# Reshape counts to match the grid for visualization
counts_matrix = counts.reshape((rows, cols))

plt.figure(figsize=(6, 5))
plt.imshow(counts_matrix, cmap='viridis', interpolation='nearest')
plt.colorbar(label='Number of times selected')
plt.title('Node Selection Frequency (100000 trials)')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

## Finding MCDS

### Greedy Algorithm Function

In [ ]:
def greedy_connected_dominating_set(G, start_node):
    dominating_set = set([start_node])
    dominated = set([start_node])
    dominated.update(adjacent_8(G, start_node))

    # While not all nodes are dominated
    while len(dominated) < G.number_of_nodes():
        # Candidates: nodes adjacent (4-way) to the dominating set, not already in it
        candidates = set()
        for node in dominating_set:
            for neighbor in adjacent_4(G, node):
                if neighbor not in dominating_set:
                    candidates.add(neighbor)
        # For each candidate, count how many new nodes it would dominate
        best_candidate = None
        max_new = -1
        for candidate in candidates:
            new_dominated = set(adjacent_8(G, candidate) + [candidate]) - dominated
            if len(new_dominated) > max_new:
                max_new = len(new_dominated)
                best_candidate = candidate
        # Add the best candidate to the dominating set
        if best_candidate is not None:
            dominating_set.add(best_candidate)
            dominated.update(adjacent_8(G, best_candidate))
            dominated.add(best_candidate)
        else:
            # If no candidate found (should not happen in a connected grid), break
            break
    return dominating_set

### Visualize Greedy Solution

In [ ]:
overlay_subset(grid_graph, greedy_connected_dominating_set(grid_graph, select_starting_node_8way_probablistic(grid_graph)))

In [ ]:
overlay_subset(grid_graph, greedy_connected_dominating_set(grid_graph, select_starting_node_max_neighbors(grid_graph)))

#### Probabilistic Greedy Algorithm Function

In [ ]:
def probabilistic_greedy_connected_dominating_set(G, start_node):
    dominating_set = set([start_node])
    dominated = set([start_node])
    dominated.update(adjacent_8(G, start_node))

    while len(dominated) < G.number_of_nodes():
        # Candidates: nodes adjacent (4-way) to the dominating set, not already in it
        candidates = set()
        for node in dominating_set:
            for neighbor in adjacent_4(G, node):
                if neighbor not in dominating_set:
                    candidates.add(neighbor)
        if not candidates:
            break  # Should not happen in a connected grid

        # For each candidate, count how many new nodes it would dominate
        candidate_list = list(candidates)
        new_dominated_counts = []
        for candidate in candidate_list:
            new_dominated = set(adjacent_8(G, candidate) + [candidate]) - dominated
            new_dominated_counts.append(len(new_dominated))

        # If all candidates would dominate 0 new nodes, break
        if sum(new_dominated_counts) == 0:
            break

        # Select a candidate with probability proportional to new_dominated_counts
        selected_candidate = random.choices(candidate_list, weights=new_dominated_counts, k=1)[0]

        dominating_set.add(selected_candidate)
        dominated.update(adjacent_8(G, selected_candidate))
        dominated.add(selected_candidate)

    return dominating_set

#### Visualize Probabilistic Greedy Function

In [ ]:
overlay_subset(grid_graph, probabilistic_greedy_connected_dominating_set(grid_graph, select_starting_node_8way_probablistic(grid_graph)))

### Comparisons

#### Comparison of Starting Node Selection Methods

In [ ]:
# Run many trials to compare average CDS size for both starting node selection methods
num_trials = 100

s1 = print_runtime_greedy("Probabilistic start greedy", select_starting_node_8way_probablistic, greedy_connected_dominating_set, grid_graph, num_trials)
s2 = print_runtime_greedy("Maximum degree start greedy", select_starting_node_max_neighbors, greedy_connected_dominating_set, grid_graph, num_trials)
s3 = print_runtime_greedy("Maximum degree start probabilistic", select_starting_node_max_neighbors, probabilistic_greedy_connected_dominating_set, grid_graph, num_trials)
s4 = print_runtime_greedy("Probabilistic start probabilistic", select_starting_node_8way_probablistic, probabilistic_greedy_connected_dominating_set, grid_graph, num_trials)

import matplotlib.pyplot as plt

data = [s1, s2, s3, s4]
labels = [
    "Probabilistic start greedy",
    "Maximum degree start greedy",
    "Maximum degree start probabilistic",
    "Probabilistic start probabilistic"
]

plt.figure(figsize=(8, 6))
plt.boxplot(data, labels=labels)
plt.ylabel("CDS Size")
plt.title("Comparison of CDS Sizes Across Methods")
plt.xticks(rotation=15)
plt.show()

In [ ]:
grid_graph = generate_2D_grid_graph(10, 10)

s1 = print_runtime_greedy("Probabilistic start greedy", select_starting_node_8way_probablistic, greedy_connected_dominating_set, grid_graph, num_trials)
s2 = print_runtime_greedy("Maximum degree start greedy", select_starting_node_max_neighbors, greedy_connected_dominating_set, grid_graph, num_trials)
s3 = print_runtime_greedy("Maximum degree start probabilistic", select_starting_node_max_neighbors, probabilistic_greedy_connected_dominating_set, grid_graph, num_trials)
s4 = print_runtime_greedy("Probabilistic start probabilistic", select_starting_node_8way_probablistic, probabilistic_greedy_connected_dominating_set, grid_graph, num_trials)

In [ ]:
grid_graph = generate_2D_grid_graph(30, 30)

print_runtime_greedy("Probabilistic start greedy", select_starting_node_8way_probablistic, greedy_connected_dominating_set, grid_graph, num_trials)
print_runtime_greedy("Maximum degree start greedy", select_starting_node_max_neighbors, greedy_connected_dominating_set, grid_graph, num_trials)
print_runtime_greedy("Maximum degree start probabilistic", select_starting_node_max_neighbors, probabilistic_greedy_connected_dominating_set, grid_graph, num_trials)
print_runtime_greedy("Probabilistic start probabilistic", select_starting_node_8way_probablistic, probabilistic_greedy_connected_dominating_set, grid_graph, num_trials)

In [ ]:
# Increase trial time latr